# Extract DTM file names from HTML table and create polygon vector layer with the tiles' BBs

The German Federal State of North Rhine-Westphalia (NRW or NW) provides wonderful and huge open data on its **[NRW Open Geodata Portal](https://www.opengeodata.nrw.de/produkte/)**.

One of the fantastic data sets is the [**Digital Terrain Model (DTM) tiles of NRW in 1m horizontal resolution**](https://www.opengeodata.nrw.de/produkte/geobasis/hm/dgm1_xyz/dgm1_xyz/). This link provides a table with the filenames and download links of 35860 DTM tiles covering NRW of 1 km² size.

To see on a map where these tiles a squared polygon is created for each tile with shows location and extent of the tiles. The coordinates of the corner of the squares are derived from the filename which contains the lower left corner and the tile size (here: 1000 x 1000 grid points with 1m spacing, yielding 1000 m x 1000 m total tile size).  

Web scraping ([Wikipedia](https://en.wikipedia.org/wiki/Web_scraping)) with [`BeautifulSoup`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) is used to extract the DTM filenames from the HTML table listing all the files [here](https://www.opengeodata.nrw.de/produkte/geobasis/hm/dgm1_xyz/dgm1_xyz/).

In [ ]:
dtm_url = r"https://www.opengeodata.nrw.de/produkte/geobasis/hm/dgm1_xyz/dgm1_xyz/"

In [ ]:
data_dir = r"../data/derived/NRW_DTM_NRW_EPSG_25832_Tiles_BB/"
out_fname = r"NRW_DTM_NRW_EPSG_25832_Tiles_BB.gpkg"

In [ ]:
import requests
from bs4 import BeautifulSoup
from shapely.geometry import Polygon
import geopandas as gpd
import os

In [ ]:
os.makedirs(data_dir,exist_ok = True)

## Exercise

Complete the function below!

In [ ]:
def points_from_fname(fname, Dx=1000, Dy=1000):
    """
    Usage: [(x_LL,y_LL),(x_UL,y_UL),(x_UR,y_UR),(x_LR,y_LR)]  = points_from_fname(...)
       
    Returns a list of four 2-tuples representing the corner points of a square. 
    Create corner points of the squared bounding box for a NRW DTM tile in ESPG:25832.
    URL of NRW DTM tile collection: https://www.opengeodata.nrw.de/produkte/geobasis/hm/dgm1_xyz/dgm1_xyz/
    The coordinates of the lower left corner are extracted from the filename which is 
    formatted like 'dgm1_32_280_5652_1_nw.xyz.gz'. Filename elements:
    dgm1: name of product with 1x1 m² grid cell size, 32: UTM Zone 32, EPSG:25832, 
    280: Easting in km, 5652: Northing in km, 1: 1m x 1m grid cell size, nw: North Rhine-Wastphalia,
    xyz: ASCII fixed width file format with three columns (easting, northing, elevation), gz: GNU zipped
    """
    
    x_Left  = [...] # Extract it from fname. Use units m, not km.
    y_Low   = [...] # Extract it from fname. Use units m, not km.
    x_Right = [...] # Add the cell size ...
    y_Up    = [...] # Add the cell size ...

    P_LL = (x_Left,y_Low) # e.g. (280000,5652000)
    P_UL = (...)
    P_UR = (...)
    P_LR = (...)
    
    return [P_LL, P_UL, P_UR, P_LR] 

In [ ]:
help(points_from_fname)

## Exercise:

Read the docs. Google.

1. How does `BeautifulSoup` work? What is a tag? Which attributes does the tag `file` have?
1. How does `requests` work?
1. How do you create a GeoPandas dataframe from a dictionary? How does this dictionary have to look like?

In [ ]:
r = requests.get(dtm_url)
soup = BeautifulSoup(r.content, 'html.parser')

In [ ]:
tag_list = soup.find_all('file')
fname_list = [tag["name"] for tag in tag_list]
geom_list = [Polygon(points_from_fname(fname)) for fname in fname_list]

In [ ]:
# fname: polygon attribute, geometry: polygon geometry 
dic = {'fname': fname_list, 'geometry': geom_list}
gdf = gpd.GeoDataFrame(data = dic, crs="EPSG:25832")

In [ ]:
print(f"gdf.shape: {gdf.shape}")
gdf.head()

In [ ]:
print(f"Write gdf to file {data_dir + out_fname:s}")
gdf.to_file(data_dir + out_fname, driver = "GPKG") 